In [329]:
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import

import os
import string
import requests
import xml.etree.ElementTree as ET

from lxml import html

import matplotlib.pyplot as plt
import numpy as np

In [318]:
def reach_in_cm(str_reach):
    if not str_reach or '-' in str_reach:
        return None
    reach_inches = float(str_reach.strip('\"'))
    return reach_inches * 2.54

In [319]:
def weight_in_kg(str_pounds):
    if not str_pounds or '-' in str_pounds:
        return None
    pounds = float(str_pounds.split(' ')[0]) 
    return pounds * 0.45359237

In [320]:
def height_in_cm(str_height):
    if not str_height or '-' in str_height:
        return None
    feet, inches = str_height.split(' ')
    feet = feet.strip('\'')
    inches = inches.strip('\"')
    inches = int(inches)
    inches += (int(feet) * 12)
    return inches * 2.54 # centimeter

In [325]:
def get_user(elements):
    fname = elements[0].xpath('a')[0].text.strip()
    lname = elements[1].xpath('a')[0].text.strip()
    nickname = elements[2].xpath('a')[0].text
    height = height_in_cm(elements[3].text.strip())
    weight = weight_in_kg(elements[4].text.strip())
    reach = reach_in_cm(elements[5].text.strip())
    stance = None if not elements[6].text.strip() else elements[6].text.strip()
    win = elements[7].text.strip()
    loss = elements[8].text.strip()
    dq = elements[9].text.strip()
    champ = len(elements[10].xpath('img')) >= 1
    stats_link = elements[0].xpath('a')[0].attrib['href']
    
    return np.array([fname, lname, nickname, 
                     height, weight, reach, 
                     stance, win, loss, dq, 
                     champ, stats_link])

In [332]:
alphabet = list(string.ascii_lowercase)
users = []

table_head = None
    

for char in alphabet:
    URL = "http://www.ufcstats.com/statistics/fighters?char={}&page=all"
    page = requests.get(url=URL.format(char)) 
    tree = html.fromstring(page.content)
    URL.format('a')
    
    # init table header
    if table_head is None:
        table_head = tree.xpath('//table/thead/tr/th/text()')
        table_head = np.array(table_head)
        for i, value in enumerate(table_head):
            table_head[i] = value.strip()
    
    table_data = tree.xpath('//table/tbody/tr')
    table_rows = np.array(table_data[1:])  

    for i, row_data in enumerate(table_rows):        
        users.append(get_user(row_data))

In [333]:
np.array(users).shape

(3393, 12)